## 第二部

- 今まで
    - 計算をモデル化してきた
        - ch2: SIMPLE
        - ch3: DFA/NFA
        - ch4: DPDA/NPDA
        - ch5: DTM
    - 計算ってなんだろう？を考えてきた
         - 質問に答えるもの
         - 入力に対し出力返すもの
    - 人間がわかりやすい形での理解だった
- こっから
    - 人間の理解しやすい形**ではない**形式で考えていく

## chapter 6
- ゼロから計算の概念を組み立てていく
- 型なしラムダ計算
    - ch6.1: rubyのprocを用いてlambda計算（っぽいこと）をする
    - ch6.2: lambda計算を実装する

## 6.1: ラムダ計算をまねる
- 最小限の機能でプログラムを書こう
- 使用禁止
    - gem
    - std lib
    - module
    - method
    - class
    - object
    - 制御構造
    - 代入
    - 配列
    - 文字列
    - 数値
    - ブール値
- 使用可能
    - 変数参照
        - aliasとしての変数利用
        - ?代入禁止とconflictするのでは
    - proc生成
    - proc呼び出し
    
- 要するに：論理構造，データ構造などすべて禁止．自前でつくっていく
- こんな感じですべてを表す

### 6.1.1: procのポイント

procってなにができるの

In [75]:
#proc
-> x { -> { x.call(y)}}

#<Proc:0x007febfc058f58@<main>:1 (lambda)>

#### 6.1.1.1 配管

callできるよ

In [76]:
# call
-> x {x + 2}.call(1)

3

#### 6.1.1.2 引数

複数引数取れるよ

In [77]:
-> x,y {x + y}.call(3,4)

7

でも別に，複数引数撮れることはmustじゃないよ

仮にprocが単引数しか許容しなかったとしても，同じことは出来るよ

In [78]:
-> x {
  -> y {
    x + y
  }
}.call(3).call(4)

7

この場合，引数を少なく返すと「残りの引数を受け取るproc」が返るよ

In [79]:
# 2引数のところ1引数で渡すと..
addThree = -> x {
  -> y {
    x + y
  }
}.call(3)

#<Proc:0x007febfd0db830@<main>:2 (lambda)>

In [80]:
# 引数に3をたす関数になった
addThree.call(4)

7

In [81]:
addThree.call(10)

13

いわゆるカリー化である

#### 6.1.1.3 等価性

内部定義が違っても，同じ呼び出しに同じ結果を返すなら等価である
外延等価性というらしい

以下は等価である

In [82]:
p = -> n {n * 2}

#<Proc:0x007febfd06adb0@<main>:0 (lambda)>

In [83]:
q = -> x { p.call(x)}

#<Proc:0x007febfd04a060@<main>:0 (lambda)>

In [84]:
puts p.call(10)
puts q.call(10)

20
20


等価であれば，交換可能であるのは自然の理

```
p = q
p = -> x { p.call(x)}
```
すなわち，

```
-> x {p.call(x)}
```
なるパターンを見つけたら無条件で `p` に書き換えてok

#### 6.1.1.4 構文

引数はこんな形でも与えられるよ

In [85]:
-> x {x + 5}[10]

15

### 6.1.2 問題

FizzBuzzを先の制約のもとかいてみよう

まずは普通にrubyつかってFizzBuzzかいて，それをstep by stepで変換していくよ

In [86]:
# いわゆるFizzBuzz
(1..100).map do |n|
  if (n % 15).zero?
    'FizzBuzz'
  elsif (n % 3).zero?
    'Fizz'
  elsif (n % 5).zero?
    'Buzz'
  else
    n
  end
end

[1, 2, "Fizz", 4, "Buzz", "Fizz", 7, 8, "Fizz", "Buzz", 11, "Fizz", 13, 14, "FizzBuzz", 16, 17, "Fizz", 19, "Buzz", "Fizz", 22, 23, "Fizz", "Buzz", 26, "Fizz", 28, 29, "FizzBuzz", 31, 32, "Fizz", 34, "Buzz", "Fizz", 37, 38, "Fizz", "Buzz", 41, "Fizz", 43, 44, "FizzBuzz", 46, 47, "Fizz", 49, "Buzz", "Fizz", 52, 53, "Fizz", "Buzz", 56, "Fizz", 58, 59, "FizzBuzz", 61, 62, "Fizz", 64, "Buzz", "Fizz", 67, 68, "Fizz", "Buzz", 71, "Fizz", 73, 74, "FizzBuzz", 76, 77, "Fizz", 79, "Buzz", "Fizz", 82, 83, "Fizz", "Buzz", 86, "Fizz", 88, 89, "FizzBuzz", 91, 92, "Fizz", 94, "Buzz", "Fizz", 97, 98, "Fizz", "Buzz"]

このコードは使っちゃいけないものばかり使っているね

- 数
- ブール値
- 制御構造(if/elsif/else)
- 演算
- リスト( (1..100))
- 文字列
- etc.

ひとつずつ，すべてprocで表現していくよ

### 6.1.3 数

- 「数」という概念を「1,2,3..」を使わずに表現するよ
- 「数」ってなんだ？
    - ０の次に１
    - 1の次に２
    - ２の次に3..
- 1とか2のラベルに意味は無い
    - 表現の形でしか無いからね
    - 本質は「連続・反復を表す概念である」ということ
    - 我々は 0->1->2 という連続性を知っているので，これらのラベルは「数」を表現できる
        - その意味では「あ，い，う，え，お」とかも一緒
        - 実際16進数だとA,B..Fつかってるよね
- 「連続・反復」の概念を表現できれば，数を表現できる
    - 0: なにもしないこと
    - 1: アクションを行うこと
    - 2: アクションを行った後，アクションを行うこと
    - 3: アクションを行った後，アクションを行った後，アクションを行うこと
    - 以下略
    - 再帰っぽいな！

In [87]:
# ruby風に数を表現
def zero (proc, x)
  x
end

def one (proc, x)
  proc[x]
end

def two (proc, x)
  proc[proc[x]]
end

:two

よーするに：引数xに対しproc p を「何度」適用するか？で数を表す

In [88]:
# procで同じことをやると
ZERO = -> p { -> x { x}}
ONE = -> p { -> x {p[x]}}
TWO = -> p { -> x {p[p[x]]}}

<main>:1: warning: already initialized constant ZERO
<main>:1: warning: previous definition of ZERO was here
<main>:2: warning: already initialized constant ONE
<main>:2: warning: previous definition of ONE was here
<main>:3: warning: already initialized constant TWO
<main>:3: warning: previous definition of TWO was here


#<Proc:0x007febfd8ccee0@<main>:3 (lambda)>

スッキリかけて気持ちいいね！でも頭がこんがらがってきそうだね！

- このままだと本当にZEROが0, ONEが1なのか？がわかりづらい
- 人間が理解しやすいように，これらの値をruby世界の「数」に変換する関数をつくろう

In [89]:
# ZERO/ONE/..を0/1/..に変換する
def to_integer(proc)
  proc[-> x { x+1}][0]
end

:to_integer

In [90]:
puts to_integer(ZERO)
puts to_integer(ONE)
puts to_integer(TWO)

0
1
2


これで，FizzBuzzにひつような数 3, 5, 15, 100が定義できる！

In [91]:
THREE = -> p { -> x {p[p[p[x]]]}}
FIVE = -> p{ -> x {p[p[p[p[p[x]]]]]}}
FIFTEEN = -> p{ -> x {p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[x]]]]]]]]]]]]]]]}}
HUNDRED = -> p{ -> x {p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[x]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]}}

<main>:1: warning: already initialized constant THREE
<main>:1: warning: previous definition of THREE was here
<main>:1: warning: already initialized constant FIVE
<main>:1: warning: previous definition of FIVE was here
<main>:2: warning: already initialized constant FIFTEEN
<main>:2: warning: previous definition of FIFTEEN was here
<main>:3: warning: already initialized constant HUNDRED
<main>:3: warning: previous definition of HUNDRED was here


#<Proc:0x007febfd988e88@<main>:3 (lambda)>

In [92]:
puts to_integer(THREE)
puts to_integer(FIVE)
puts to_integer(FIFTEEN)
puts to_integer(HUNDRED)

3
5
15
100


以上を用いてFizzBuzzはこうなります

In [93]:
#  数をprocで表現
(ONE..HUNDRED).map do |n|
  if (n % FIFTEEN).zero?
    'FizzBuzz'
  elsif (n % THREE).zero?
    'Fizz'
  elsif (n % FIVE).zero?
    'Buzz'
  else
    n
  end
end

ArgumentError: bad value for range

ただし，まだ独自の「数」に対して % とか .. とかを定義していないので，上のプログラムは動きません

### 6.1.4 ブール値

- ブール値ってなんだ？
    - True/False
    - ２つの選択肢から一つを選ぶもの
        - if bool then true_statement else false_statement
    - 最初から true_statement, false_statement のどちらかを返すもの，と定義できる

In [94]:
# rubyふうに
def true(x,y)
  x
end

def false(x,y)
  y
end

:false

In [95]:
# procふうに
# TRUE/FALSEは予約語っぽいorz
MY_TRUE = -> x { -> y { x }}
MY_FALSE = -> x { -> y { y }}

<main>:2: warning: already initialized constant MY_TRUE
<main>:2: warning: previous definition of MY_TRUE was here
<main>:3: warning: already initialized constant MY_FALSE
<main>:3: warning: previous definition of MY_FALSE was here


#<Proc:0x007febfd8f6bf0@<main>:3 (lambda)>

変換も簡単

In [96]:
def to_boolean(proc)
  proc[true][false]
end

:to_boolean

In [97]:
puts to_boolean(MY_TRUE)
puts to_boolean(MY_FALSE)

true
false


合せて制御構造 if/elsif/else も定義しよう

In [98]:
# ruby ふうに
def if_(proc, x, y)
  proc[x][y]
end

:if_

In [99]:
# procふうに
IF = -> b { -> x { -> y { b[x][y]}}}

<main>:1: warning: already initialized constant IF
<main>:1: warning: previous definition of IF was here


#<Proc:0x007febfc092f28@<main>:1 (lambda)>

ここで procの変換ルールを思い出そう

`-> x { p[x]} ` == `p` だった

これをIFに適用

```
-> b {-> x {-> y { b[x][y]}}}
-> b {-> x {        b[x]      }}
-> b {              b          }}
```

そもそも true/falseを「２つの選択肢から一つを選ぶもの」として定義したので，ごく当たり前の話ではあるね

In [100]:
# 数，bool値をprocで
(ONE..HUNDRED).map do |n|
  IF[(n % FIFTEEN).zero?][
    'FizzBuzz'
  ][IF[(n % THREE).zero?][
    'Fizz'
  ][IF[(n % FIVE).zero?][
    'Buzz'
  ][
    n
  ]]]
end

ArgumentError: bad value for range

### 6.1.5 述語

- FizzBuzzをつくるためには，余りがゼロかの判定が必要（.zero?）
- 我々は「数」を「アクションを行う回数」と定義した
- 「常にfalse返すproc」をtrueに何度も適用する，とかんがえる

In [101]:
# rubyふう
def zreo?(proc)
  proc[->x{FALSE}][TRUE]
end

:zreo?

In [102]:
# procふう
IS_ZERO = -> p { p[ -> x {MY_FALSE}][MY_TRUE]}

<main>:1: warning: already initialized constant IS_ZERO
<main>:1: warning: previous definition of IS_ZERO was here


#<Proc:0x007febfd856060@<main>:1 (lambda)>

In [103]:
puts to_boolean( IS_ZERO[ZERO])
puts to_boolean( IS_ZERO[ONE])

true
false


In [104]:
# 数,bool値,述語をprocで
(ONE..HUNDRED).map do |n|
  IF[IS_ZERO[(n % FIFTEEN)]][
    'FizzBuzz'
  ][IF[IS_ZERO[(n % THREE)]][
    'Fizz'
  ][IF[IS_ZERO[(n % FIVE)]][
    'Buzz'
  ][
    n
  ]]]
end

ArgumentError: bad value for range

### 6.1.6 ペア
- プリミティブとして数値，ブール値を定義した
- データ構造が欲しくなる
- ペアを定義する

- 要件
    - 2つの要素をもつ
    - left で1つ目の要素を取得
    - right で2つ目の要素を取得

In [105]:
# procふうに
PAIR = -> x { -> y { -> f { f[x][y] }}}
LEFT = -> p { p[ -> x { -> y {x}}]} 
RIGHT = -> p { p[-> x { -> y {y}}]} 

<main>:1: warning: already initialized constant PAIR
<main>:1: warning: previous definition of PAIR was here
<main>:2: warning: already initialized constant LEFT
<main>:2: warning: previous definition of LEFT was here
<main>:3: warning: already initialized constant RIGHT
<main>:3: warning: previous definition of RIGHT was here


#<Proc:0x007febfd81d440@<main>:3 (lambda)>

パッと見難しいけど，落ち着いて考えればやってることは簡単

- pair : procを受け取り，「予め受け取っていた二数x,y」を適用するproc
- left: procを受け取り，「二数x,yを受け取りxを返すproc」を適用するproc
- right: procを受け取り，「二数x,yを受け取りyを返すproc」を適用するproc

### 6.1.7 数値演算

- 各種データ型がそろってきた
- 演算を定義する

- まずはインクリメント
- 数 = p,x を受け取ってxにpを指定回数適用するもの
- インクリメント: nに更にpを適用する

In [106]:
INCREMENT = -> n { -> p { -> x {p[n[p][x]]}}}

<main>: warning: already initialized constant INCREMENT


#<Proc:0x007febfd069050@<main>:0 (lambda)>

- デクリメント
- n回適用された時，一回戻ったものを返す
    - むずかしい
- pairを利用，slideという概念の導入


In [107]:
def slide(pair)
  [pair.last, pair.last+1]
end

# 一つずつずれたものになる
puts slide([3,4])
puts slide([7,8])

# ぶっちゃけ重要なの後ろの数だけ
puts slide([0, 4])
puts slide([10000, 4])

[4, 5]
[8, 9]
[4, 5]
[4, 5]


- 数n: xにpをn回適用するもの
- [0,0]にslideをn回適用する -> [n-1, n]
- left取ればdecrementできてる！

In [108]:
SLIDE = -> p {PAIR[RIGHT[p]][INCREMENT[RIGHT[p]]]}
DECREMENT = -> n { LEFT[n[SLIDE][PAIR[ZERO][ZERO]]]}

SyntaxError: <main>:1: syntax error, unexpected end-of-input, expecting '}'
DECREMENT = -> n { LEFT[n[SLIDE][PAIR[ZERO][ZERO]]]
                                                   ^

In [109]:
puts to_integer(DECREMENT[ONE])
puts to_integer(DECREMENT[FIVE])
puts to_integer(DECREMENT[HUNDRED])

0
4
99


- 以上を用いれば四則演算が定義できるよ
- カリー化してることに注目するとわかりやすいかも

In [142]:
ADD = -> m { -> n { n[INCREMENT][m] }}
SUBTRACT = -> m { -> n {n[DECREMENT][m]}}
MULTIPLY = -> m { -> n {n[ADD[m]][ZERO]}}
POWER = -> m { -> n { n[MULTIPLY[m]][ONE]}}

<main>:1: warning: already initialized constant ADD
<main>:1: warning: previous definition of ADD was here
<main>:1: warning: already initialized constant SUBTRACT
<main>:1: warning: previous definition of SUBTRACT was here
<main>:2: warning: already initialized constant MULTIPLY
<main>:2: warning: previous definition of MULTIPLY was here
<main>:3: warning: already initialized constant POWER
<main>:3: warning: previous definition of POWER was here


#<Proc:0x007febfda7a508@<main>:3 (lambda)>

In [111]:
# こう考えるとわかりやすいかも

# 「5を足す」というproc
ADD_FIVE = FIVE[INCREMENT]

# nを受け「nを足す」というprocを返すproc
ADD_N = -> n {n[INCREMENT]}

#  mにnを受け「nを足す」というprocを返すprocを適用

ADD_N_TO_M = -> m { -> n {ADD_N[n][m]}}

<main>:3: warning: already initialized constant ADD_FIVE
<main>:3: warning: previous definition of ADD_FIVE was here
<main>:6: warning: already initialized constant ADD_N
<main>:6: warning: previous definition of ADD_N was here
<main>:9: warning: already initialized constant ADD_N_TO_M
<main>:9: warning: previous definition of ADD_N_TO_M was here


#<Proc:0x007febfc873558@<main>:9 (lambda)>

In [144]:
# +α

# add，これでよくね？
MYADD = -> m {-> n { -> p { -> x {m[p][n[p][x]]}}}}
puts to_integer(MYADD[TWO][FIVE])

# mul,　これでよくね？
MYMUL = -> m {-> n { -> p {m[n[p]]}}}
puts to_integer(MYMUL[TWO][FIVE])

<main>:3: warning: already initialized constant MYADD
<main>:1: warning: previous definition of MYADD was here


7


<main>:7: warning: already initialized constant MYMUL
<main>:5: warning: previous definition of MYMUL was here


10


Haskell風に書くとこんなんとか

```
ZERO = \x -> x

```

- あとは剰余（%）がほしい

In [112]:
def my_mod(m,n)
  if n <= m
    mod(m-n, n)
  else
    m
  end
end

:my_mod

- 割る数nを割られる数mから引きまくって，mが以下になったらreturnする
- なんてことはない
- 比較演算子(<=)を定義できれば行けそうだ!

```
m <= n
m - n <= 0
```

- 数m-nと0の比較ができれば，<=を定義できたことになる
- 我々の「数」においては，ゼロ以下の数は存在しない
    - decrement の定義より
- すなわち「aがゼロ以下」のとき，「a==0」となる
- すでにIS_ZEROは定義済みなので，<=が定義できそうだ

In [113]:
# m <= n
IS_LESS_OR_EQUAL = -> m {-> n { IS_ZERO[SUBTRACT[m][n]]}}

<main>:1: warning: already initialized constant IS_LESS_OR_EQUAL
<main>:1: warning: previous definition of IS_LESS_OR_EQUAL was here


#<Proc:0x007febfc86bad8@<main>:1 (lambda)>

In [114]:
puts to_boolean(IS_LESS_OR_EQUAL[FIVE][ONE])
puts to_boolean(IS_LESS_OR_EQUAL[ONE][FIVE])

false
true


- IS_LESS_OR_EQUALを用いてMODを定義しよう

In [115]:
MOD = -> m { -> n {
    IF
    [IS_LESS_OR_EQUAL[n][m]]
    [MOD[SUBTRACT[m][n]][n]]
    [m]
}}

<main>: warning: already initialized constant MOD


#<Proc:0x007febfd94fed0@<main>:0 (lambda)>

- ただしこのままでは動かない
- rubyのifは遅延評価だが．IFは正格評価のため
    - cond評価の前にthen/else節を評価する
    - 無限にthen節が呼ばれる
    - `MOD[SUBTRACT[ZERO][n]][n]` の繰り返しである
- then 節を遅延させたい
    - procでくるめばok
    - 等価性を思い出す
    - `p` = `->x {p[x]}`
    

In [136]:
MYMOD = -> m { -> n {
    IF[IS_LESS_OR_EQUAL[n][m]][
      -> x {
        MYMOD[SUBTRACT[m][n]][n][x]
        }
      ][
        m
        ]
}}

<main>: warning: already initialized constant MYMOD


#<Proc:0x007febfda92c98@<main>:0 (lambda)>

In [137]:
puts to_boolean(IS_LESS_OR_EQUAL[FIVE][TWO])
puts to_integer(MYMOD[THREE][TWO])

false
1


- このコードはMYMODを変数として再帰のために利用している
- 代入の機能をつかっているではないか！
- 再帰関数定義のためのprocがひつよう
- 通称Yコンビネータ
     - よーするに：fを無限に繰り返します
- 手を動かして考えたほうがわかると思う

In [138]:
Y = -> f { -> x { f[x[x]] }[-> x { f[x[x]] }] }

#<Proc:0x007febfd1d95c0@<main>:0 (lambda)>

In [139]:
Z = -> f { -> x { f[-> y { x[x][y] }] }[-> x { f[-> y { x[x][y] }] }] }

#<Proc:0x007febfc181178@<main>:0 (lambda)>